In [22]:
import numpy as np 
import scipy.optimize as opt
import matplotlib.pyplot as plot
import os 
import time
####This program has some problems associated with it, so I need to go through this to discover what's wrong. 

In [20]:
def Load_Data_Files():
# =============================================================================
# This section of the code retrieves all the data files held within the folder
# of interest and then orders them by sub-folder, and if they are 
# experimental data files or experimental parameter files
# =============================================================================
    # Set the folder position in the file directory
    folder_directory = 'I:\\Science\\Physics-Nanobiophotonics\\Group\\User Data\\ODMR\\ODMR and imaging\\20200821'
    date_position = folder_directory.find('2020')
    date = folder_directory[date_position:date_position+8]

    # Then the files for each folder are sorted into designated varibales. 
    file_directory = []
     
    for files in os.listdir(folder_directory):
        #print(files)
        time.sleep(1)
        file_directory.append(folder_directory +  '\\' + files)
        
    
    file_directory = np.array(file_directory)
    return(file_directory, date)  


In [3]:
def Sort_Data_Files(all_files):
    data_files = []
    params_files = []
    
    # Sorts all the files from Load_Data_Files into those containing experiment
    # data and those containing the scan parameters. 
    for i in range(len(all_files[:,])):
        if all_files[i,].find('NV') != -1:
            if all_files[i,].find('Params') != -1:
                params_files.append(all_files[i,])
            else:
                data_files.append(all_files[i,])
        else: 
            continue
    
    return(np.array(data_files), np.array(params_files))

In [4]:
def Analyse_CW_Data(data_files, params_files, date):
    # This is the master control portion of the code, it determines which bits 
    # of analysis are performed on the data files. 
    for i in range(len(data_files[:,])):
        data_file_name = data_files[i,]
        data = Load_Data(data_file_name)
        mean_data = Mean_Data(data)
        
        normalised_data, fluorescent_baseline = Normalise_Data(mean_data)
        freq = Get_Freq_Data(params_files[i,])
        fig1, ax1 = scatter_plot(freq, normalised_data)
        fig2 = colour_plot(freq, data)
        
        peak_fittings = Peak_Fittings(freq, normalised_data, fluorescent_baseline, ax1)
        save_data(date, data_file_name, peak_fittings, fig1, fig2)
        time.sleep(0.5)
        
        print(data_file_name) 
        
        plot.close(fig1)
        plot.close(fig2)
    
    return(peak_fittings)

In [5]:
def Load_Data(data_file):
    data = np.loadtxt(data_file)
    return(data)

In [6]:
def Mean_Data(data):
    mean_data = np.mean(data, axis = 0)
    return(mean_data)
    

In [7]:
def Get_Freq_Data(params_file):
    params = np.loadtxt(params_file, delimiter = '\t', usecols = 1)
    start = params[0]
    stop = params[1]
    no_of_steps = int(np.floor(1 + ((params[1] - params[0]))/(params[2]*0.001)))
    # Creates a variable containing the applied frequencies. 
    freq = np.linspace(start, stop, no_of_steps)

    return(freq)

In [19]:
def Normalise_Data(mean_data):
    # normalises the mean data to a mean of the first 10% of datapoint recorded. 
    fluorescent_baseline = np.mean(mean_data[0:int(len(mean_data)/10)])
    normalised_data = mean_data#/fluorescent_baseline
    return(normalised_data, fluorescent_baseline)

In [9]:
def scatter_plot(freq,mean_data):
    # Creates a scatter plot for experimental data and plot parameters. 
    fig1, ax1 = plot.subplots()
    ax1.scatter(freq, mean_data, marker = 'x',color = 'r', label = "mean ODMR linescan data")
#    plot.scatter(freq, mean_data, marker = 'o',color = 'b', label = "Low resolution ODMR")
    ax1.set_xlabel('Microwave Frequency  (GHz)')
    ax1.set_ylabel('Counts (per int. time)')
    ax1 = plot.gca()
    ax1.set_xlim(np.min(freq), np.max(freq))
    ax1.set_ylim(np.min(mean_data)*0.97, np.max(mean_data)*1.01)
    ax1.legend(loc = 'lower right')
    return(fig1, ax1)

In [10]:
def colour_plot(freq, data):
    # A plot of all the data recorded in a 2D heatmap (not normalised so drift 
    # affected the results. 
    
    Y = np.linspace(1, len(data[:,0]), len(data[:,0])) 
    # Y = Variable for the number of scans completed to take dataset. 

    fig2 = plot.figure()
    plot.pcolormesh(freq, Y, data, cmap = 'viridis')
    plot.colorbar()
    plot.xlim(np.min(freq),np.max(freq))
    plot.xticks(np.arange(np.min(freq), np.max(freq), 0.05))
    plot.xlabel(r'$\nu$  (GHz)')
    plot.ylabel('Number of scans')
    return(fig2)

In [11]:
def get_fluorescent_minima(mean_data):
    # A function that finds the two minima of the ODMR data.
    mid_point = int(len(mean_data)/2)
    minima = []
    minima.append(min(mean_data[0:mid_point]))
    minima.append(min(mean_data[mid_point+1 : len(mean_data)]))
    
    return (minima, mid_point)

In [12]:
def get_res_frequency(mean_data, mid_point, minima, freq):
    # Using the fluorescent minima poisitons, an approximation of the 
    # gaussian dip positions is made
    
    res_freq = np.empty([1,len(minima)])
    for i in range(len(minima)):
        min_arr_pos = np.where(mean_data[i*mid_point:i*mid_point+mid_point] == minima[i])
        a = min_arr_pos[0]+i*mid_point
        b = freq[a]
        res_freq[0,i] = b
    return (res_freq)

In [13]:
def gauss_curve_fitting(freq, mean_data, cf_parameters):
    # Finds the optimal fitting parameters and the associated error for a 
    # double Gauss curve fitting. 
    
    optimised_cf_params, confidence = opt.curve_fit(Double_Gauss_Fit, 
                                        freq, mean_data, cf_parameters)
    
    cf_errors = np.sqrt(np.diag(confidence))
    
    return(optimised_cf_params, cf_errors) 

In [14]:
def lorentz_curve_fitting(freq, mean_data, cf_parameters):
    # Finds the optimal fitting parameters and the associated error for a 
    # double Lorentzian curve fitting. 
    
    optimised_cf_params, confidence = opt.curve_fit(Double_Lorentz_Fit, 
                                        freq, mean_data, cf_parameters)
    
    cf_errors = np.sqrt(np.diag(confidence))
    
    return(optimised_cf_params, cf_errors)   

In [15]:
def Double_Gauss_Fit(freq, amp1, res_freq1, fwhm1, amp2, res_freq2, fwhm2, baseline):
    y1 = -1* amp1 * np.exp(-1 * ( (res_freq1-freq)**2) / (fwhm1**2) )
    y2 = -1* amp2 * np.exp(-1 * ( (res_freq2-freq)**2) / (fwhm2**2) )
    
    gauss_fit = y1 + y2 + baseline
    return(gauss_fit)

In [16]:
def Peak_Fittings(freq, mean_data, fluorescent_baseline, ax1):
    # A function that finds the two minima of the ODMR data and then generates 
    # either a two peak lorenztian or gaussian fitting (user discretion) for 
    # the data.
   
    minimas, mid_point = get_fluorescent_minima(mean_data)
    
    dip_amplitude = [fluorescent_baseline - minimas[0],
                     fluorescent_baseline - minimas[1] ]
    
    res_freq = get_res_frequency(mean_data, mid_point, minimas, freq)
   # print(res_freq)
        
    line_width = [0.001, 0.001] # Approximation for the FWHM of the applied 
                                # line fit. 
    
    cf_parameters = [dip_amplitude[0], res_freq[0,0], line_width[0], 
                     dip_amplitude[1], res_freq[0,1], line_width[1], 
                     fluorescent_baseline]    
    
    #print(cf_parameters)
    
    optimised_cf_params, cf_errors = gauss_curve_fitting(freq, mean_data, 
                                                         cf_parameters)


    cf_fit_array = []
    for i in range(len(optimised_cf_params)):
        cf_fit_array.append(optimised_cf_params[i])
        cf_fit_array.append(cf_errors[i])
    
   
    curve_fit = Double_Gauss_Fit(freq, optimised_cf_params[0], 
                                   optimised_cf_params[1], 
                                 optimised_cf_params[2], optimised_cf_params[3], 
                                 optimised_cf_params[4], optimised_cf_params[5], 
                                 optimised_cf_params[6])
    
    ax1.plot(freq, curve_fit, color = 'k', label='Gaussian Fit')
    ax1.legend()
    
    return(cf_fit_array) 

In [17]:
# =============================================================================
# Saving the data gathered
# =============================================================================
def save_data(date, file_ID, peak_fittings, fig1, fig2):
    
    destination_folder = 'C:\\Users\\Ryan\\Desktop\\2020821_ODMR' + '\\'
    output_data_file = destination_folder + date + '_ODMR_analysis.dat'
    # Checking to see if the folder exists in which to save the data, and creating it if it does not
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)
        # Checking the output file exists and creating and initialising it if not
    if os.path.isfile(output_data_file) == False:
        with open(output_data_file, "w") as wf:
            wf.write('File name' +  ', \t' + 'Amplitude 1 (counts)' +  '\t' 
                     + r'\$delta$ Amplitude 1' + '\t' + 'Resonant frequency 1 (GHz)' + 
                     '\t' + r'\$delta$ Resonant frequency 1 (GHz)' + '\t' + 
                     'FWHM Peak 1 (GHz)' + '\t' + r'\$delta$ FMHW Peak 1' + '\t'
                     + 'Amplitude 2 (counts)' +  '\t' + r'\$delta$ Amplitude 2'
                     + '\t' + 'Resonant frequency 2 (GHz)' + '\t' + 
                     r'\$delta$ Resonant frequency 2 (GHz)' + '\t' + 
                     'FWHM Peak 2 (GHz)' + '\t' + r'\$delta$ FMHW Peak 2' + '\t'
                     + 'Fluorescent Baseline (counts per int time)' + '\t' + 
                     r'\$delta$ Fluorescent Baseline' + '\n')
    # Writing contrast dip description to the .dat file
    
    with open(output_data_file, "a") as wf:        
        wf.write(file_ID + '\t')
        for i in range(len(peak_fittings)):
            wf.write(str(round(peak_fittings[i],6)) + '\t')
        wf.write('\n')
#   
    
    find_file_name = file_ID.find('Scan')
    find_dot = file_ID.find('.')
    short_file_name = file_ID[find_file_name + 6 : find_dot]

#    # Saving a plot to a folder. 
    fig1.savefig(destination_folder + short_file_name + '_scatter.png', transparent = True)    
    fig2.savefig(destination_folder + short_file_name + '_heatmap.png', transparent = True)
   

In [21]:
all_files, date = Load_Data_Files()

data_files, params_files = Sort_Data_Files(all_files)
ana = Analyse_CW_Data(data_files, params_files, date)

I:\Science\Physics-Nanobiophotonics\Group\User Data\ODMR\ODMR and imaging\20200821\20200821_NV_Scan_01.dat
I:\Science\Physics-Nanobiophotonics\Group\User Data\ODMR\ODMR and imaging\20200821\20200821_NV_Scan_02.dat
I:\Science\Physics-Nanobiophotonics\Group\User Data\ODMR\ODMR and imaging\20200821\20200821_NV_Scan_03.dat
I:\Science\Physics-Nanobiophotonics\Group\User Data\ODMR\ODMR and imaging\20200821\20200821_NV_Scan_04.dat
I:\Science\Physics-Nanobiophotonics\Group\User Data\ODMR\ODMR and imaging\20200821\20200821_NV_Scan_05.dat
I:\Science\Physics-Nanobiophotonics\Group\User Data\ODMR\ODMR and imaging\20200821\20200821_NV_Scan_06.dat
I:\Science\Physics-Nanobiophotonics\Group\User Data\ODMR\ODMR and imaging\20200821\20200821_NV_Scan_07.dat
I:\Science\Physics-Nanobiophotonics\Group\User Data\ODMR\ODMR and imaging\20200821\20200821_NV_Scan_08.dat
I:\Science\Physics-Nanobiophotonics\Group\User Data\ODMR\ODMR and imaging\20200821\20200821_NV_Scan_09.dat
I:\Science\Physics-Nanobiophotonics\G

C:\Users\Ryan\AppData\Local\conda\conda\envs\Ryan\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in sqrt
  


I:\Science\Physics-Nanobiophotonics\Group\User Data\ODMR\ODMR and imaging\20200821\20200821_NV_Scan_12.dat
I:\Science\Physics-Nanobiophotonics\Group\User Data\ODMR\ODMR and imaging\20200821\20200821_NV_Scan_13.dat
I:\Science\Physics-Nanobiophotonics\Group\User Data\ODMR\ODMR and imaging\20200821\20200821_NV_Scan_14.dat
I:\Science\Physics-Nanobiophotonics\Group\User Data\ODMR\ODMR and imaging\20200821\20200821_NV_Scan_15.dat
I:\Science\Physics-Nanobiophotonics\Group\User Data\ODMR\ODMR and imaging\20200821\20200821_NV_Scan_16.dat
I:\Science\Physics-Nanobiophotonics\Group\User Data\ODMR\ODMR and imaging\20200821\20200821_NV_Scan_17.dat
I:\Science\Physics-Nanobiophotonics\Group\User Data\ODMR\ODMR and imaging\20200821\20200821_NV_Scan_18.dat
I:\Science\Physics-Nanobiophotonics\Group\User Data\ODMR\ODMR and imaging\20200821\20200821_NV_Scan_19.dat
I:\Science\Physics-Nanobiophotonics\Group\User Data\ODMR\ODMR and imaging\20200821\20200821_NV_Scan_20.dat
I:\Science\Physics-Nanobiophotonics\G